In [8]:
!git clone https://github.com/anantSinghCross/xray_classification_pneumonia.git

fatal: destination path 'xray_classification_pneumonia' already exists and is not an empty directory.


In [9]:
%pwd

'c:\\Workspace\\medical1\\p0604-13'

In [10]:
import glob
base_path = './xray_classification_pneumonia/Dataset_compressed/train'
file_path = list(glob.glob(base_path + "/*/*.*"))

pneumonia = list(glob.glob(base_path+"/PNEUMONIA/*.*"))
normal = list(glob.glob(base_path+"/NORMAL/*.*"))

In [11]:
len(pneumonia), len(normal)

(3875, 1341)

In [12]:
import matplotlib.pyplot as plt
import os

test = os.path.dirname(file_path[0])
print(test)
class_name = os.path.split(os.path.dirname(test))
print(class_name)

./xray_classification_pneumonia/Dataset_compressed/train\NORMAL
('./xray_classification_pneumonia/Dataset_compressed', 'train')


In [13]:
# folder name to label
labels = []

for fp in file_path:
    tmp = os.path.dirname(fp)
    class_name = os.path.split(tmp)
    if class_name[1] == "PNEUMONIA":
        labels.append(1)
    else:
        labels.append(0)

In [14]:
import numpy as np
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2


img_width = 60
img_height = 60

dataset = np.ndarray(shape=(len(file_path), img_height*img_width), dtype=np.float32)

i=0
for _file in file_path:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (img_width, img_height))
    
    dataset[i] = img_resized.flatten()
    i += 1
    if i % 250 == 0:
        print(f"{i} images to array")
print("Done!")

250 images to array
500 images to array
750 images to array
1000 images to array
1250 images to array
1500 images to array
1750 images to array
2000 images to array
2250 images to array
2500 images to array
2750 images to array
3000 images to array
3250 images to array
3500 images to array
3750 images to array
4000 images to array
4250 images to array
4500 images to array
4750 images to array
5000 images to array
Done!


In [15]:
# custom - 계산을 안정적으로 처리
def sigmoid(x):
    pos_mask = (x>=0)
    neg_mask = (x<0)
    z = np.zeros_like(x) # 입력 값과 같은 크기의 배열을 0으로 생성
    z[pos_mask] = np.exp(-x[pos_mask])
    z[neg_mask] = np.exp(x[neg_mask])
    top = np.ones_like(x)
    top[neg_mask] = z[neg_mask] #입력 값이 0보다 작은 경우 분자를 exp(x) 설정
    return top / (1+z) 
    

In [16]:
# softmax
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

In [17]:
def init_network():
    network = {}
    network['W1'] = np.random.randn(3600*100).reshape((3600, 100))
    network['b1'] = np.random.randn(100)
    network['W2'] = np.random.randn(100*50).reshape((100, 50))
    network['b2'] = np.random.randn(50)
    network['W3'] = np.random.randn(50*2).reshape((50, 2))
    network['b3'] = np.random.randn(2)
    
    return network

def identify_function(x):
    return x

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y


### Prediction

In [18]:
x, t = dataset, labels
network = init_network()

accuracy_cnt = 0

for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy: " + str(float(accuracy_cnt/len(x))))
    

Accuracy: 0.7429064417177914
